<a href="https://colab.research.google.com/github/afizrahman/Global-Electronics-Sales-Analysis/blob/main/DEVELOP_1O_SQL_QUERY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 17.8 MB/s eta 0:00:00


In [2]:
!pip install pandas

In [3]:
import pandas as pd

In [6]:
# TOP 10 BEST SELLING PRODUCTS
!pip install mysql-connector-python pandas SQLAlchemy pymysql


import pandas as pd
from sqlalchemy import create_engine


user = "3a4M49L7Ack2pZS.root"
password = "HeTIP3fdzSUFLm1X"
host = "gateway01.ap-northeast-1.prod.aws.tidbcloud.com"
port = 4000
database = "dataspark"


connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?ssl_ca=<CA_PATH>&ssl_verify_identity=True"
engine = create_engine(connection_string)

print(" Connected to TiDB Cloud!")

check_cols = pd.read_sql("SHOW COLUMNS FROM sales;", con=engine)
print("\n Full Columns in 'sales' table:")
display(check_cols)


query = """
SELECT
    p.product_name,
    p.category,
    SUM(s.Quantity) AS total_units_sold
FROM dataspark.sales s
JOIN dataspark.products p
    ON s.productkey = p.productkey
GROUP BY p.product_name, p.category
ORDER BY total_units_sold DESC
LIMIT 10;
"""


try:
    df = pd.read_sql(query, con=engine)
    print("\n Query executed successfully!\n")
    display(df.head())
except Exception as e:
    print("Query failed. Error message:")
    print(e)

 Connected to TiDB Cloud!

 Full Columns in 'sales' table:


,Field,Type,Null,Key,Default,Extra
0,Order_Number,int(11),YES,,None,
1,Line_Item,int(11),YES,,None,
2,Order_Date,varchar(255),YES,,None,
3,Delivery_Date,varchar(255),YES,,None,
4,CustomerKey,int(11),YES,,None,
5,StoreKey,int(11),YES,,None,
6,ProductKey,int(11),YES,,None,
7,Quantity,int(11),YES,,None,
8,Currency_Code,varchar(255),YES,,None,



 Query executed successfully!



,product_name,category,total_units_sold
0,WWI Desktop PC2.33 X2330 Black,Computers,550.0
1,WWI Desktop PC1.80 E1800 White,Computers,538.0
2,Adventure Works Desktop PC2.30 MD230 White,Computers,521.0
3,Adventure Works Desktop PC1.60 ED160 Black,Computers,521.0
4,Adventure Works Desktop PC1.80 ED180 Black,Computers,520.0


In [10]:
# TOTAL REVENUE BY COUNTRY
!pip install mysql-connector-python pandas SQLAlchemy pymysql


import pandas as pd
from sqlalchemy import create_engine


user = "3a4M49L7Ack2pZS.root"
password = "HeTIP3fdzSUFLm1X"
host = "gateway01.ap-northeast-1.prod.aws.tidbcloud.com"
port = 4000
database = "dataspark"


connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?ssl_ca=<CA_PATH>&ssl_verify_identity=True"
engine = create_engine(connection_string)

print(" Connected to TiDB Cloud!")

check_cols_sales = pd.read_sql("SHOW COLUMNS FROM sales;", con=engine)
print("\n Full Columns in 'sales' table:")
display(check_cols_sales)

check_cols_products = pd.read_sql("SHOW COLUMNS FROM products;", con=engine)
print("\n Full Columns in 'products' table:")
display(check_cols_products)


query = """
SELECT
    c.country,
    SUM(s.Quantity * p.Unit_Price_USD) AS total_revenue_usd
FROM dataspark.sales s
JOIN dataspark.customer c
    ON s.customerkey = c.customerid
JOIN dataspark.products p
    ON s.productkey = p.productkey
GROUP BY c.country
ORDER BY total_revenue_usd DESC;
"""


try:
    df = pd.read_sql(query, con=engine)
    print("\n Query executed successfully!\n")
    display(df.head())
except Exception as e:
    print("Query failed. Error message:")
    print(e)

 Connected to TiDB Cloud!

 Full Columns in 'sales' table:


,Field,Type,Null,Key,Default,Extra
0,Order_Number,int(11),YES,,None,
1,Line_Item,int(11),YES,,None,
2,Order_Date,varchar(255),YES,,None,
3,Delivery_Date,varchar(255),YES,,None,
4,CustomerKey,int(11),YES,,None,
5,StoreKey,int(11),YES,,None,
6,ProductKey,int(11),YES,,None,
7,Quantity,int(11),YES,,None,
8,Currency_Code,varchar(255),YES,,None,



 Full Columns in 'products' table:


,Field,Type,Null,Key,Default,Extra
0,ProductKey,int(11),YES,,None,
1,Product_Name,varchar(255),YES,,None,
2,Brand,varchar(255),YES,,None,
3,Color,varchar(255),YES,,None,
4,Category,varchar(255),YES,,None,
5,Unit_Cost_USD,float,YES,,None,
6,Unit_Price_USD,float,YES,,None,
7,Subcategory,varchar(255),YES,,None,
8,CategoryKey,varchar(255),YES,,None,



 Query executed successfully!



,country,total_revenue_usd
0,United States,2.987163e+07
1,United Kingdom,7.084088e+06
2,Germany,5.414150e+06
3,Canada,4.724335e+06
4,Australia,2.708138e+06


In [18]:
# AVERAGE ORDER VALUE
!pip install mysql-connector-python pandas SQLAlchemy pymysql


import pandas as pd
from sqlalchemy import create_engine


user = "3a4M49L7Ack2pZS.root"
password = "HeTIP3fdzSUFLm1X"
host = "gateway01.ap-northeast-1.prod.aws.tidbcloud.com"
port = 4000
database = "dataspark"


connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?ssl_ca=<CA_PATH>&ssl_verify_identity=True"
engine = create_engine(connection_string)

print(" Connected to TiDB Cloud!")

check_cols = pd.read_sql("SHOW COLUMNS FROM sales;", con=engine)
print("\n Full Columns in 'sales' table:")
display(check_cols)


query = """
WITH order_values AS (
    SELECT
        s.Order_Number,
        s.StoreKey,
        SUM(s.Quantity * p.Unit_Price_USD) AS order_total
    FROM dataspark.sales s
    JOIN dataspark.products p ON s.ProductKey = p.ProductKey
    GROUP BY s.Order_Number, s.StoreKey
)
SELECT
    st.state,
    st.storekey,
    AVG(ov.order_total) AS avg_order_value
FROM order_values ov
JOIN dataspark.stores st ON ov.StoreKey = st.storekey
GROUP BY st.state, st.storekey
ORDER BY avg_order_value DESC;
"""


try:
    df = pd.read_sql(query, con=engine)
    print("\n Query executed successfully!\n")
    display(df.head())
except Exception as e:
    print("Query failed. Error message:")
    print(e)

 Connected to TiDB Cloud!

 Full Columns in 'sales' table:


,Field,Type,Null,Key,Default,Extra
0,Order_Number,int(11),YES,,None,
1,Line_Item,int(11),YES,,None,
2,Order_Date,varchar(255),YES,,None,
3,Delivery_Date,varchar(255),YES,,None,
4,CustomerKey,int(11),YES,,None,
5,StoreKey,int(11),YES,,None,
6,ProductKey,int(11),YES,,None,
7,Quantity,int(11),YES,,None,
8,Currency_Code,varchar(255),YES,,None,



 Query executed successfully!



,state,storekey,avg_order_value
0,Northern Territory,2,3793.997504
1,Mayotte,18,2791.097284
2,Saarland,26,2716.257861
3,Tasmania,4,2542.959875
4,Pesaro,30,2450.437262


In [11]:
# MONTHLY SALES TREND
!pip install mysql-connector-python pandas SQLAlchemy pymysql


import pandas as pd
from sqlalchemy import create_engine


user = "3a4M49L7Ack2pZS.root"
password = "HeTIP3fdzSUFLm1X"
host = "gateway01.ap-northeast-1.prod.aws.tidbcloud.com"
port = 4000
database = "dataspark"


connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?ssl_ca=<CA_PATH>&ssl_verify_identity=True"
engine = create_engine(connection_string)

print(" Connected to TiDB Cloud!")

check_cols = pd.read_sql("SHOW COLUMNS FROM sales;", con=engine)
print("\n Full Columns in 'sales' table:")
display(check_cols)


query = """
SELECT
    DATE_FORMAT(s.Order_Date, '%%Y-%%m-01') AS month,
    SUM(s.Quantity) AS monthly_quantity
FROM dataspark.sales s
GROUP BY month
ORDER BY month;

"""

try:
    df = pd.read_sql(query, con=engine)
    print("\n Query executed successfully!\n")
    display(df.head())
except Exception as e:
    print("Query failed. Error message:")
    print(e)

 Connected to TiDB Cloud!

 Full Columns in 'sales' table:


,Field,Type,Null,Key,Default,Extra
0,Order_Number,int(11),YES,,None,
1,Line_Item,int(11),YES,,None,
2,Order_Date,varchar(255),YES,,None,
3,Delivery_Date,varchar(255),YES,,None,
4,CustomerKey,int(11),YES,,None,
5,StoreKey,int(11),YES,,None,
6,ProductKey,int(11),YES,,None,
7,Quantity,int(11),YES,,None,
8,Currency_Code,varchar(255),YES,,None,



 Query executed successfully!



,month,monthly_quantity
0,2016-01-01,2116.0
1,2016-02-01,2676.0
2,2016-03-01,895.0
3,2016-04-01,326.0
4,2016-05-01,1646.0


In [12]:
# REVENUE BY PRODUCT CATEGORY
!pip install mysql-connector-python pandas SQLAlchemy pymysql


import pandas as pd
from sqlalchemy import create_engine


user = "3a4M49L7Ack2pZS.root"
password = "HeTIP3fdzSUFLm1X"
host = "gateway01.ap-northeast-1.prod.aws.tidbcloud.com"
port = 4000
database = "dataspark"


connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?ssl_ca=<CA_PATH>&ssl_verify_identity=True"
engine = create_engine(connection_string)

print(" Connected to TiDB Cloud!")

check_cols = pd.read_sql("SHOW COLUMNS FROM sales;", con=engine)
print("\n Full Columns in 'sales' table:")
display(check_cols)


query = """
SELECT
    p.category,
    SUM(s.Quantity) AS total_quantity_sold
FROM dataspark.sales s
JOIN dataspark.products p
    ON s.productkey = p.productkey
GROUP BY p.category
ORDER BY total_quantity_sold DESC;

"""

try:
    df = pd.read_sql(query, con=engine)
    print("\n Query executed successfully!\n")
    display(df.head())
except Exception as e:
    print("Query failed. Error message:")
    print(e)

 Connected to TiDB Cloud!

 Full Columns in 'sales' table:


,Field,Type,Null,Key,Default,Extra
0,Order_Number,int(11),YES,,None,
1,Line_Item,int(11),YES,,None,
2,Order_Date,varchar(255),YES,,None,
3,Delivery_Date,varchar(255),YES,,None,
4,CustomerKey,int(11),YES,,None,
5,StoreKey,int(11),YES,,None,
6,ProductKey,int(11),YES,,None,
7,Quantity,int(11),YES,,None,
8,Currency_Code,varchar(255),YES,,None,



 Query executed successfully!



,category,total_quantity_sold
0,Computers,44151.0
1,Cell phones,31477.0
2,"Music, Movies and Audio Books",28802.0
3,Audio,23490.0
4,Games and Toys,22591.0


In [13]:
# CUSTOMER SEGMENTATION BY SPENDING

!pip install mysql-connector-python pandas SQLAlchemy pymysql


import pandas as pd
from sqlalchemy import create_engine


user = "3a4M49L7Ack2pZS.root"
password = "HeTIP3fdzSUFLm1X"
host = "gateway01.ap-northeast-1.prod.aws.tidbcloud.com"
port = 4000
database = "dataspark"


connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?ssl_ca=<CA_PATH>&ssl_verify_identity=True"
engine = create_engine(connection_string)

print(" Connected to TiDB Cloud!")

check_cols_sales = pd.read_sql("SHOW COLUMNS FROM sales;", con=engine)
print("\n Full Columns in 'sales' table:")
display(check_cols_sales)

check_cols_customer = pd.read_sql("SHOW COLUMNS FROM customer;", con=engine)
print("\n Full Columns in 'customer' table:")
display(check_cols_customer)


query = """
SELECT
    c.CustomerID, -- Placeholder, as customer_segment is unknown
    COUNT(DISTINCT s.customerkey) AS total_customers,
    SUM(s.Quantity) AS total_quantity_sold
FROM dataspark.sales s
JOIN dataspark.customer c
    ON s.customerkey = c.customerid
GROUP BY c.CustomerID -- Group by CustomerID until a segmentation column is found
ORDER BY total_quantity_sold DESC;

"""

try:
    df = pd.read_sql(query, con=engine)
    print("\n Query executed successfully!\n")
    display(df.head())
except Exception as e:
    print("Query failed. Error message:")
    print(e)

 Connected to TiDB Cloud!

 Full Columns in 'sales' table:


,Field,Type,Null,Key,Default,Extra
0,Order_Number,int(11),YES,,None,
1,Line_Item,int(11),YES,,None,
2,Order_Date,varchar(255),YES,,None,
3,Delivery_Date,varchar(255),YES,,None,
4,CustomerKey,int(11),YES,,None,
5,StoreKey,int(11),YES,,None,
6,ProductKey,int(11),YES,,None,
7,Quantity,int(11),YES,,None,
8,Currency_Code,varchar(255),YES,,None,



 Full Columns in 'customer' table:


,Field,Type,Null,Key,Default,Extra
0,CustomerID,int(11),NO,PRI,None,
1,Gender,varchar(255),YES,,None,
2,Name,varchar(255),YES,,None,
3,City,varchar(255),YES,,None,
4,State,varchar(255),YES,,None,
5,Country,varchar(255),YES,,None,
6,Continent,varchar(255),YES,,None,
7,Birthday,varchar(255),YES,,None,



 Query executed successfully!



,CustomerID,total_customers,total_quantity_sold
0,723572,1,124.0
1,1925694,1,123.0
2,461982,1,107.0
3,1736139,1,103.0
4,1579183,1,100.0


In [20]:
# CURRENCY IMPACT ON REVENUE
!pip install mysql-connector-python pandas SQLAlchemy pymysql


import pandas as pd
from sqlalchemy import create_engine


user = "3a4M49L7Ack2pZS.root"
password = "HeTIP3fdzSUFLm1X"
host = "gateway01.ap-northeast-1.prod.aws.tidbcloud.com"
port = 4000
database = "dataspark"


connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?ssl_ca=<CA_PATH>&ssl_verify_identity=True"
engine = create_engine(connection_string)

print(" Connected to TiDB Cloud!")

check_cols_sales = pd.read_sql("SHOW COLUMNS FROM sales;", con=engine)
print("\n Full Columns in 'sales' table:")
display(check_cols_sales)

check_cols_customer = pd.read_sql("SHOW COLUMNS FROM customer;", con=engine)
print("\n Full Columns in 'customer' table:")
display(check_cols_customer)


query = """
SELECT
    e.currency_code,
    AVG(e.exchange) AS avg_exchange_rate,
    SUM(s.Quantity * p.Unit_Price_USD / e.exchange) AS total_revenue_local
FROM dataspark.sales s
JOIN dataspark.products p
    ON s.productkey = p.productkey
JOIN dataspark.exchange e
    ON s.Currency_Code = e.currency_code
GROUP BY e.currency_code
ORDER BY total_revenue_local DESC;
"""


try:
    df = pd.read_sql(query, con=engine)
    print("\n Query executed successfully!\n")
    display(df.head())
except Exception as e:
    print("Query failed. Error message:")
    print(e)

 Connected to TiDB Cloud!

 Full Columns in 'sales' table:


,Field,Type,Null,Key,Default,Extra
0,Order_Number,int(11),YES,,None,
1,Line_Item,int(11),YES,,None,
2,Order_Date,varchar(255),YES,,None,
3,Delivery_Date,varchar(255),YES,,None,
4,CustomerKey,int(11),YES,,None,
5,StoreKey,int(11),YES,,None,
6,ProductKey,int(11),YES,,None,
7,Quantity,int(11),YES,,None,
8,Currency_Code,varchar(255),YES,,None,



 Full Columns in 'customer' table:


,Field,Type,Null,Key,Default,Extra
0,CustomerID,int(11),NO,PRI,None,
1,Gender,varchar(255),YES,,None,
2,Name,varchar(255),YES,,None,
3,City,varchar(255),YES,,None,
4,State,varchar(255),YES,,None,
5,Country,varchar(255),YES,,None,
6,Continent,varchar(255),YES,,None,
7,Birthday,varchar(255),YES,,None,



 Query executed successfully!



,currency_code,avg_exchange_rate,total_revenue_local
0,USD,1.000000,6.700207e+10
1,EUR,0.883785,2.889419e+10
2,GBP,0.747151,2.137616e+10
3,CAD,1.310171,8.097041e+09
4,AUD,1.367305,4.456345e+09


In [14]:
# STORE PERFOMANCE BY SIZE

!pip install mysql-connector-python pandas SQLAlchemy pymysql


import pandas as pd
from sqlalchemy import create_engine


user = "3a4M49L7Ack2pZS.root"
password = "HeTIP3fdzSUFLm1X"
host = "gateway01.ap-northeast-1.prod.aws.tidbcloud.com"
port = 4000
database = "dataspark"


connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?ssl_ca=<CA_PATH>&ssl_verify_identity=True"
engine = create_engine(connection_string)

print(" Connected to TiDB Cloud!")

check_cols_sales = pd.read_sql("SHOW COLUMNS FROM sales;", con=engine)
print("\n Full Columns in 'sales' table:")
display(check_cols_sales)

check_cols_customer = pd.read_sql("SHOW COLUMNS FROM customer;", con=engine)
print("\n Full Columns in 'customer' table:")
display(check_cols_customer)


query = """
SELECT
    st.Country,
    CASE
        WHEN st.square_metres < 500 THEN 'Small'
        WHEN st.square_metres BETWEEN 500 AND 1500 THEN 'Medium'
        ELSE 'Large'
    END AS store_size,
    COUNT(DISTINCT st.Storekey) AS Num_Stores,
    SUM(s.Quantity) AS Total_Quantity
FROM dataspark.stores st
JOIN dataspark.sales s
    ON st.storekey = s.Storekey
GROUP BY st.Country, Store_Size
ORDER BY Total_Quantity DESC;

"""

try:
    df = pd.read_sql(query, con=engine)
    print("\n Query executed successfully!\n")
    display(df.head())
except Exception as e:
    print("Query failed. Error message:")
    print(e)

 Connected to TiDB Cloud!

 Full Columns in 'sales' table:


,Field,Type,Null,Key,Default,Extra
0,Order_Number,int(11),YES,,None,
1,Line_Item,int(11),YES,,None,
2,Order_Date,varchar(255),YES,,None,
3,Delivery_Date,varchar(255),YES,,None,
4,CustomerKey,int(11),YES,,None,
5,StoreKey,int(11),YES,,None,
6,ProductKey,int(11),YES,,None,
7,Quantity,int(11),YES,,None,
8,Currency_Code,varchar(255),YES,,None,



 Full Columns in 'customer' table:


,Field,Type,Null,Key,Default,Extra
0,CustomerID,int(11),NO,PRI,None,
1,Gender,varchar(255),YES,,None,
2,Name,varchar(255),YES,,None,
3,City,varchar(255),YES,,None,
4,State,varchar(255),YES,,None,
5,Country,varchar(255),YES,,None,
6,Continent,varchar(255),YES,,None,
7,Birthday,varchar(255),YES,,None,



 Query executed successfully!



,Country,store_size,Num_Stores,Total_Quantity
0,United States,Large,12,52189.0
1,Online,Large,1,41311.0
2,United States,Medium,8,31449.0
3,United Kingdom,Large,5,14113.0
4,Canada,Medium,2,8803.0


In [15]:
# AGE GROUP SPENDING BEHAVIOUR

!pip install mysql-connector-python pandas SQLAlchemy pymysql


import pandas as pd
from sqlalchemy import create_engine


user = "3a4M49L7Ack2pZS.root"
password = "HeTIP3fdzSUFLm1X"
host = "gateway01.ap-northeast-1.prod.aws.tidbcloud.com"
port = 4000
database = "dataspark"


connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?ssl_ca=<CA_PATH>&ssl_verify_identity=True"
engine = create_engine(connection_string)

print(" Connected to TiDB Cloud!")

check_cols_sales = pd.read_sql("SHOW COLUMNS FROM sales;", con=engine)
print("\n Full Columns in 'sales' table:")
display(check_cols_sales)

check_cols_customer = pd.read_sql("SHOW COLUMNS FROM customer;", con=engine)
print("\n Full Columns in 'customer' table:")
display(check_cols_customer)


query = """
SELECT
    c.CustomerID, -- Age_Group column not found in customer table, using CustomerID as placeholder.
    COUNT(DISTINCT s.customerkey) AS Total_Customers,
    SUM(s.Quantity) AS Total_Quantity_Sold -- Total_Revenue not found, using Quantity
FROM dataspark.sales s
JOIN dataspark.customer c
    ON s.customerkey = c.customerid
GROUP BY c.CustomerID -- Group by CustomerID as Age_Group is not present
ORDER BY Total_Quantity_Sold DESC;
"""

try:
    df = pd.read_sql(query, con=engine)
    print("\n Query executed successfully!\n")
    display(df.head())
except Exception as e:
    print("Query failed. Error message:")
    print(e)

 Connected to TiDB Cloud!

 Full Columns in 'sales' table:


,Field,Type,Null,Key,Default,Extra
0,Order_Number,int(11),YES,,None,
1,Line_Item,int(11),YES,,None,
2,Order_Date,varchar(255),YES,,None,
3,Delivery_Date,varchar(255),YES,,None,
4,CustomerKey,int(11),YES,,None,
5,StoreKey,int(11),YES,,None,
6,ProductKey,int(11),YES,,None,
7,Quantity,int(11),YES,,None,
8,Currency_Code,varchar(255),YES,,None,



 Full Columns in 'customer' table:


,Field,Type,Null,Key,Default,Extra
0,CustomerID,int(11),NO,PRI,None,
1,Gender,varchar(255),YES,,None,
2,Name,varchar(255),YES,,None,
3,City,varchar(255),YES,,None,
4,State,varchar(255),YES,,None,
5,Country,varchar(255),YES,,None,
6,Continent,varchar(255),YES,,None,
7,Birthday,varchar(255),YES,,None,



 Query executed successfully!



,CustomerID,Total_Customers,Total_Quantity_Sold
0,723572,1,124.0
1,1925694,1,123.0
2,461982,1,107.0
3,1736139,1,103.0
4,1579183,1,100.0


In [16]:
# REPEAT PURCHASE RATE
!pip install mysql-connector-python pandas SQLAlchemy pymysql


import pandas as pd
from sqlalchemy import create_engine


user = "3a4M49L7Ack2pZS.root"
password = "HeTIP3fdzSUFLm1X"
host = "gateway01.ap-northeast-1.prod.aws.tidbcloud.com"
port = 4000
database = "dataspark"


connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?ssl_ca=<CA_PATH>&ssl_verify_identity=True"
engine = create_engine(connection_string)

print(" Connected to TiDB Cloud!")

check_cols_sales = pd.read_sql("SHOW COLUMNS FROM sales;", con=engine)
print("\n Full Columns in 'sales' table:")
display(check_cols_sales)

check_cols_customer = pd.read_sql("SHOW COLUMNS FROM customer;", con=engine)
print("\n Full Columns in 'customer' table:")
display(check_cols_customer)


query = """
WITH customer_orders AS (
    SELECT
        customerkey,
        COUNT(DISTINCT order_number) AS order_count
    FROM dataspark.sales
    GROUP BY customerkey
)
SELECT
    SUM(CASE WHEN order_count > 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS repeat_customer_pct
FROM customer_orders;

"""

try:
    df = pd.read_sql(query, con=engine)
    print("\n Query executed successfully!\n")
    display(df.head())
except Exception as e:
    print("Query failed. Error message:")
    print(e)

 Connected to TiDB Cloud!

 Full Columns in 'sales' table:


,Field,Type,Null,Key,Default,Extra
0,Order_Number,int(11),YES,,None,
1,Line_Item,int(11),YES,,None,
2,Order_Date,varchar(255),YES,,None,
3,Delivery_Date,varchar(255),YES,,None,
4,CustomerKey,int(11),YES,,None,
5,StoreKey,int(11),YES,,None,
6,ProductKey,int(11),YES,,None,
7,Quantity,int(11),YES,,None,
8,Currency_Code,varchar(255),YES,,None,



 Full Columns in 'customer' table:


,Field,Type,Null,Key,Default,Extra
0,CustomerID,int(11),NO,PRI,None,
1,Gender,varchar(255),YES,,None,
2,Name,varchar(255),YES,,None,
3,City,varchar(255),YES,,None,
4,State,varchar(255),YES,,None,
5,Country,varchar(255),YES,,None,
6,Continent,varchar(255),YES,,None,
7,Birthday,varchar(255),YES,,None,



 Query executed successfully!



,repeat_customer_pct
0,61.17607
